In [1]:
# Downloading raw data required for training 

!gdown https://drive.google.com/uc?id=1E6LIfRf3UDcIDCgl8ZBI2j0odmvS5c-w

Downloading...
From: https://drive.google.com/uc?id=1E6LIfRf3UDcIDCgl8ZBI2j0odmvS5c-w
To: /content/kaggle_transfer_learning_covid.zip
105MB [00:01, 67.2MB/s] 


In [2]:
# Unzip the the file on the local instance

!unzip -qq kaggle_transfer_learning_covid.zip

In [1]:
# Import the necessary libraries
import os
import cv2 as cv
import pathlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
#### YOUR CODE HERE ####


In [2]:
# Path of the dataset
data_path = 'C:/Windows/System32/ML_PATH/Electrothon 3.0/kaggle_transfer_learning_covid - Copy/'
train_dir = os.path.join(data_path,'train')
test_dir = os.path.join(data_path, 'test')


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.applications import DenseNet201

In [4]:
# Get your train and test data
train_datagen=ImageDataGenerator(rotation_range=20,width_shift_range=0.3,height_shift_range=0.3,shear_range=0.2,preprocessing_function=preprocess_input,validation_split=0.2)
train_generator=train_datagen.flow_from_directory(train_dir,target_size=(224,224),class_mode='categorical',subset='training',shuffle=True)
val_generator=train_datagen.flow_from_directory(train_dir,target_size=(224,224),class_mode='categorical',subset='validation',batch_size=3,shuffle=True)

Found 262 images belonging to 3 classes.
Found 63 images belonging to 3 classes.


In [5]:
base_model=DenseNet201(input_shape=[224,224,3],weights='imagenet',include_top=False) 

In [6]:
x=base_model.output
base_model.trainable=False
x=keras.layers.GlobalAveragePooling2D()(x)
#x=keras.layers.Dense(512,activation='relu')(x)
preds=keras.layers.Dense(3,activation='softmax')(x) 
model=keras.models.Model(inputs=[base_model.input],outputs=[preds]) #specify the inputs and outputs

In [7]:
model.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adam(lr=0.001),metrics=['accuracy'])

In [8]:
step_size_train=train_generator.n//train_generator.batch_size
step_size_validation=val_generator.n//val_generator.batch_size 

model.fit(train_generator,validation_data=val_generator,steps_per_epoch=step_size_train,validation_steps=step_size_validation,epochs=10)

Epoch 1/10
8/8 [==============================] - 55s 7s/step - loss: 1.1054 - accuracy: 0.4870 - val_loss: 1.0286 - val_accuracy: 0.5556
Epoch 2/10
8/8 [==============================] - 50s 6s/step - loss: 0.7795 - accuracy: 0.7261 - val_loss: 1.1555 - val_accuracy: 0.5714
Epoch 3/10
8/8 [==============================] - 49s 6s/step - loss: 0.5603 - accuracy: 0.7913 - val_loss: 1.1085 - val_accuracy: 0.5397
Epoch 4/10
8/8 [==============================] - 49s 6s/step - loss: 0.4113 - accuracy: 0.8870 - val_loss: 1.2027 - val_accuracy: 0.5714
Epoch 5/10
8/8 [==============================] - 49s 6s/step - loss: 0.3993 - accuracy: 0.9000 - val_loss: 1.1982 - val_accuracy: 0.5873
Epoch 6/10
8/8 [==============================] - 51s 6s/step - loss: 0.3460 - accuracy: 0.9062 - val_loss: 1.2729 - val_accuracy: 0.5873
Epoch 7/10
8/8 [==============================] - 48s 6s/step - loss: 0.3436 - accuracy: 0.9043 - val_loss: 1.3421 - val_accuracy: 0.5714
Epoch 8/10
8/8 [==================

In [9]:
model.fit(train_generator,validation_data=val_generator,steps_per_epoch=step_size_train,validation_steps=step_size_validation,initial_epoch=10,epochs=20)

Epoch 11/20
8/8 [==============================] - 50s 6s/step - loss: 0.2513 - accuracy: 0.9348 - val_loss: 1.5693 - val_accuracy: 0.5714
Epoch 12/20
8/8 [==============================] - 50s 6s/step - loss: 0.2209 - accuracy: 0.9522 - val_loss: 1.4998 - val_accuracy: 0.5873
Epoch 13/20
8/8 [==============================] - 49s 6s/step - loss: 0.2326 - accuracy: 0.9391 - val_loss: 1.5044 - val_accuracy: 0.5714
Epoch 14/20
8/8 [==============================] - 49s 6s/step - loss: 0.2322 - accuracy: 0.9565 - val_loss: 1.5694 - val_accuracy: 0.5714
Epoch 15/20
8/8 [==============================] - 49s 6s/step - loss: 0.1997 - accuracy: 0.9522 - val_loss: 1.6095 - val_accuracy: 0.5714
Epoch 16/20
8/8 [==============================] - 49s 6s/step - loss: 0.1762 - accuracy: 0.9565 - val_loss: 1.6215 - val_accuracy: 0.5873
Epoch 17/20
8/8 [==============================] - 53s 7s/step - loss: 0.1973 - accuracy: 0.9609 - val_loss: 1.5141 - val_accuracy: 0.5873
Epoch 18/20
8/8 [==========

In [102]:
sub=pd.read_csv('kaggle_sample.csv')

In [103]:
sub.head()

,image,class
0,image1.jpeg,NaN
1,image2.jpeg,NaN
2,image3.jpeg,NaN
3,image4.jpeg,NaN
4,image5.jpeg,NaN


In [88]:
images=list(sub['image'])
test_list=[]
for image in images:
  arr=cv.imread(test_dir+'/'+image)
  arr=cv.resize(arr,(224,224),interpolation=cv.INTER_LINEAR)
  arr=preprocess_input(arr)
  test_list.append(arr)

In [89]:
test_arr=np.array(test_list)

In [104]:
predictions_probab=model.predict(test_arr)

In [105]:
predictions_label=np.argmax(predictions_probab,axis=1)

In [106]:
predictions_label

array([0, 1, 2, 0, 2, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2,
       2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 2, 0, 2, 2, 2, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2,
       2, 2, 2, 2, 1, 0, 0, 2])

In [107]:
sub['class']=predictions_label

In [108]:
def category(x):
  if x==0:
    return 'covid'
  elif x==1:
    return 'normal'
  else:
    return 'pneumonia'

In [109]:
sub['class']=sub['class'].apply(category)

In [110]:
sub.head()

,image,class
0,image1.jpeg,covid
1,image2.jpeg,normal
2,image3.jpeg,pneumonia
3,image4.jpeg,covid
4,image5.jpeg,pneumonia


In [111]:
sub.to_csv('submissions.csv',index=False)